In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json

import time
import random

In [3]:
pd.options.display.max_columns= None

In [4]:
df = pd.read_csv('nfl_data.csv')
df

,season,team,Week,Day,Date,Unnamed: 3,Unnamed: 4,OT,Unnamed: 6,Opp,Tm,Opp.1,Cmp,Att,Yds,TD,Int,Sk,Yds.1,Y/A,NY/A,Cmp%,Rate,Att.1,Yds.2,Y/A.1,TD.1,FGM,FGA,XPM,XPA,Pnt,Yds.3,3DConv,3DAtt,4DConv,4DAtt,ToP,Week.1,Day.1,Date.1,Unnamed: 3.1,Unnamed: 4.1,OT.1,Unnamed: 6.1,Opp.2,Tm.1,Opp.1.1,Cmp.1,Att.2,Yds.4,TD.2,Int.1,Sk.1,Yds.1.1,Y/A.2,NY/A.1,Cmp%.1,Rate.1,Att.1.1,Yds.2.1,Y/A.1.1,TD.1.1,FGM.1,FGA.1,XPM.1,XPA.1,Pnt.1,Yds.3.1,3DConv.1,3DAtt.1,4DConv.1,4DAtt.1,ToP.1
0,2000,CRD,1,Sun,September 3,boxscore,L,NaN,@,New York Giants,16,21,28,49,312,2,3,1,6,6.5,6.2,57.1,64.3,20,43,2.2,0,1,1,1,1,5,210,6,15,0,2,28:30,1,Sun,September 3,boxscore,L,NaN,@,New York Giants,16,21,17,25,172,0,1,0,0,6.9,6.9,68.0,70.7,41,223,5.4,3,0,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,boxscore,W,NaN,NaN,Dallas Cowboys,32,31,18,24,224,2,0,2,19,10.1,8.6,75.0,131.2,33,98,3.0,1,4,4,2,2,2,94,7,12,0,0,31:20,2,Sun,September 10,boxscore,W,NaN,NaN,Dallas Cowboys,32,31,24,34,240,3,0,1,3,7.1,6.9,70.6,119.7,22,90,4.1,0,1,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,boxscore,L,NaN,NaN,Green Bay Packers,3,29,21,43,181,0,4,1,8,4.4,4.1,48.8,21.6,13,28,2.2,0,1,1,0,0,6,288,6,14,0,1,23:32,4,Sun,September 24,boxscore,L,NaN,NaN,Green Bay Packers,3,29,18,32,279,1,0,1,9,9.0,8.5,56.3,95.7,36,176,4.9,1,5,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,boxscore,L,NaN,@,San Francisco 49ers,20,27,24,41,239,1,0,0,0,5.8,5.8,58.5,83.3,24,126,5.3,1,2,3,2,2,4,176,3,13,2,3,27:50,5,Sun,October 1,boxscore,L,NaN,@,San Francisco 49ers,20,27,22,33,215,1,0,1,5,6.7,6.3,66.7,94.9,32,130,4.1,2,2,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,boxscore,W,NaN,NaN,Cleveland Browns,29,21,17,30,169,2,0,1,2,5.7,5.5,56.7,95.0,31,146,4.7,1,3,3,2,2,3,144,2,10,1,2,31:03,6,Sun,October 8,boxscore,W,NaN,NaN,Cleveland Browns,29,21,16,22,136,0,0,1,2,6.3,5.9,72.7,88.4,31,104,3.4,3,0,1,3,3,7,206,5,14,0,1,28:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11801,2022,WAS,13,Sun,December 4,boxscore,T,OT,@,New York Giants,20,20,27,41,246,2,0,5,29,6.7,5.3,65.9,98.2,36,165,4.6,0,2,3,2,2,6,265,3,14,1,1,41:11,13,Sun,December 4,boxscore,T,OT,@,New York Giants,20,20,25,31,182,1,0,4,18,6.5,5.2,80.6,101.9,30,134,4.5,1,2,3,2,2,7,324,4,13,0,0,28:49
11802,2022,WAS,15,Sun,December 18,boxscore,L,NaN,NaN,New York Giants,12,20,17,29,228,1,0,3,21,8.6,7.1,58.6,95.2,26,159,6.1,0,2,2,0,1,4,177,1,10,1,2,29:30,15,Sun,December 18,boxscore,L,NaN,NaN,New York Giants,12,20,21,32,160,0,0,0,0,5.0,5.0,65.6,77.6,30,128,4.3,1,2,2,2,2,5,213,2,10,1,1,30:30
11803,2022,WAS,16,Sat,December 24,boxscore,L,NaN,@,San Francisco 49ers,20,37,25,34,270,3,1,2,19,8.5,7.5,73.5,113.6,33,79,2.4,0,0,0,2,2,4,192,7,13,0,2,33:35,16,Sat,December 24,boxscore,L,NaN,@,San Francisco 49ers,20,37,15,22,218,2,1,3,16,10.6,8.7,68.2,111.6,26,153,5.9,2,3,3,4,4,2,80,4,11,1,2,26:25
11804,2022,WAS,17,Sun,January 1,boxscore,L,NaN,NaN,Cleveland Browns,10,24,16,28,124,0,3,3,19,5.1,4.0,57.1,28.6,37,136,3.7,1,1,1,1,1,2,90,7,16,2,4,33:32,17,Sun,January 1,boxscore,L,NaN,NaN,Cleveland Browns,10,24,9,18,155,3,0,5,14,9.4,6.7,50.0,119.2,30,146,4.9,0,1,1,3,3,4,187,4,11,1,2,26:28


In [5]:
#Columns to be deleted
#df['Unnamed: 3'] / columns [38:48]

df.drop(df.columns[38:48], axis=1, inplace=True)

df.drop(columns= ['Unnamed: 3'], axis = 1, inplace=True)

In [6]:
# Columns to be renamed
#df['Unnamed; 4', 'Unnamed: 6']

df.columns = df.columns.str.lower()

df.rename(columns={'unnamed: 4': 'result','unnamed: 6': 'host', 'tm':'pts','opp.1':'opp_pts',
                   'cmp':'pass_cmp', 'att':'pass_att','yds':'pass_yds','td':'pass_td','yds.1':'yds_lost_sks',
                   'y/a':'yds_pass_att', 'ny/a':'net_yds_pass_att', 'rate':'pass_rating', 'att.1':'rush_att',
                   'yds.2': 'rush_yds', 'y/a.1':'rush_yds_att','td.1':'rush_td','yds.3':'punt_yds'}, inplace = True)



# List of original column names
renamed_columns = ['pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int', 'sk', 'yds_lost_sks', 'yds_pass_att', 
                    'net_yds_pass_att', 'cmp%', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_att', 
                    'rush_td', 'fgm', 'fga', 'xpm', 'xpa', 'pnt', 'punt_yds', '3dconv', '3datt', 
                    '4dconv', '4datt', 'top']

# Add '_opp' to each name
new_column_names = [col + '_opp' for col in renamed_columns]

# Renaming columns in the DataFrame
df.rename(columns=dict(zip(df.columns[37:], new_column_names)), inplace = True)

In [7]:
df.columns

Index(['season', 'team', 'week', 'day', 'date', 'result', 'ot', 'host', 'opp',
       'pts', 'opp_pts', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int',
       'sk', 'yds_lost_sks', 'yds_pass_att', 'net_yds_pass_att', 'cmp%',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_att', 'rush_td', 'fgm',
       'fga', 'xpm', 'xpa', 'pnt', 'punt_yds', '3dconv', '3datt', '4dconv',
       '4datt', 'top', 'pass_cmp_opp', 'pass_att_opp', 'pass_yds_opp',
       'pass_td_opp', 'int_opp', 'sk_opp', 'yds_lost_sks_opp',
       'yds_pass_att_opp', 'net_yds_pass_att_opp', 'cmp%_opp',
       'pass_rating_opp', 'rush_att_opp', 'rush_yds_opp', 'rush_yds_att_opp',
       'rush_td_opp', 'fgm_opp', 'fga_opp', 'xpm_opp', 'xpa_opp', 'pnt_opp',
       'punt_yds_opp', '3dconv_opp', '3datt_opp', '4dconv_opp', '4datt_opp',
       'top_opp'],
      dtype='object')

In [8]:
df['host'].isnull().sum()

np.int64(5903)

In [11]:
df['host'].value_counts()

host
away    5903
home    5903
Name: count, dtype: int64

In [128]:
df['host'] = df['host'].fillna('home')
df['host'] = df['host'].replace('@','away')

#easier way to write same piece of code as above
df['host'] = df['host'].apply(lambda x: 'away' if x == '@' else 'home')

In [19]:
#replace nan values with yes or no values accoriding to ot results
df['ot'] = df['ot'].apply(lambda x: 'yes' if x == 'OT' else ' no')

In [20]:
df['ot'].value_counts()

ot
 no    11082
yes      724
Name: count, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11806 entries, 0 to 11805
Data columns (total 63 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                11806 non-null  int64  
 1   team                  11806 non-null  object 
 2   week                  11806 non-null  int64  
 3   day                   11806 non-null  object 
 4   date                  11806 non-null  object 
 5   result                11806 non-null  object 
 6   ot                    11806 non-null  object 
 7   host                  11806 non-null  object 
 8   opp                   11806 non-null  object 
 9   pts                   11806 non-null  int64  
 10  opp_pts               11806 non-null  int64  
 11  pass_cmp              11806 non-null  int64  
 12  pass_att              11806 non-null  int64  
 13  pass_yds              11806 non-null  int64  
 14  pass_td               11806 non-null  int64  
 15  int                

In [ ]:
def clean_data(csv_data):

    # read in data in form of csv data
    df = pd.read_csv(csv_data)
    
    #drop repeated columns information
    df.drop(df.columns[38:48], axis=1, inplace=True)
    
    #drop unnecessary column of 'boxscore'
    df.drop(columns= ['Unnamed: 3'], axis = 1, inplace=True)
    
    #lowercase all column names
    df.columns = df.columns.str.lower()
    
    #rename column names
    df.rename(columns={'unnamed: 4': 'result','unnamed: 6': 'host', 'tm':'pts','opp.1':'opp_pts',
                       'cmp':'pass_cmp', 'att':'pass_att','yds':'pass_yds','td':'pass_td','yds.1':'yds_lost_sks',
                       'y/a':'yds_pass_att', 'ny/a':'net_yds_pass_att', 'rate':'pass_rating', 'att.1':'rush_att',
                       'yds.2': 'rush_yds', 'y/a.1':'rush_yds_att','td.1':'rush_td','yds.3':'punt_yds'}, inplace = True)



    # List of original column names
    renamed_columns = ['pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'int', 'sk', 'yds_lost_sks', 'yds_pass_att', 
                        'net_yds_pass_att', 'cmp%', 'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_att', 
                        'rush_td', 'fgm', 'fga', 'xpm', 'xpa', 'pnt', 'punt_yds', '3dconv', '3datt', 
                        '4dconv', '4datt', 'top']

    # Add '_opp' to each name
    new_column_names = [col + '_opp' for col in renamed_columns]

    # Renaming columns in the DataFrame
    df.rename(columns=dict(zip(df.columns[37:], new_column_names)), inplace = True)
    
    #replace nan values with home and @ with away to show which are home games or away
    df['host'] = df['host'].apply(lambda x: 'away' if x == '@' else 'home')
    
    #replace nan values with yes or no values accoriding to ot results
    df['ot'] = df['ot'].apply(lambda x: 'yes' if x == 'OT' else ' no')
    
    return df